In [8]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.


In [9]:
import nltk
import random
import os
import ssl
import streamlit as st

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [11]:
ssl._create_default_https_context = ssl._create_unverified_context

In [12]:
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ALPHA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
intents = [
    # Basic Greetings & Conversations
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you?", "What's up?", "Yo!", "Howdy!", "Good morning", "Good evening"],
        "responses": ["Hey there! Ready to talk crypto?", "Hello! Let’s dive into the blockchain vibes.", "Hey! How’s your crypto stash doing?", "Yo! What’s cooking in your wallet?", "Good morning! Hope your portfolio’s green today!"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care", "Catch you later", "Adios", "Peace out"],
        "responses": ["Catch you later! Keep your keys safe.", "See ya! May your trades be profitable.", "Take care! Don’t get rekt out there.", "Adios! Stay decentralized!", "Peace out! HODL strong."]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it", "Many thanks"],
        "responses": ["No worries! Happy to help with crypto stuff.", "You’re welcome! Keep stacking those sats.", "Anytime! Let’s keep the blockchain rolling.", "Glad I could assist! Crypto’s better with friends."]
    },

    # Cryptocurrency-Specific Intents
    {
        "tag": "crypto_basics",
        "patterns": ["What is cryptocurrency?", "How does crypto work?", "Why use cryptocurrency?", "Tell me about crypto"],
        "responses": [
            "Crypto is digital cash powered by blockchain—no banks, just code and freedom!",
            "It works with a network of computers keeping a public ledger called the blockchain. Pretty slick, right?",
            "People dig crypto for fast transfers, privacy, and betting on the future. It’s money 2.0!"
        ]
    },
    {
        "tag": "blockchain_technology",
        "patterns": ["What is blockchain?", "How does blockchain work?", "Why is blockchain cool?"],
        "responses": [
            "Blockchain’s like a tamper-proof notebook shared across tons of computers—super secure!",
            "It records transactions in blocks, links them up, and no one can mess with it once it’s locked in.",
            "It’s cool because it cuts out middlemen and keeps things transparent. Crypto wouldn’t exist without it!"
        ]
    },
    {
        "tag": "popular_cryptocurrencies",
        "patterns": ["What are the big cryptos?", "Tell me about Bitcoin", "What’s Ethereum?", "Which crypto’s hot?"],
        "responses": [
            "Big players are Bitcoin (the OG), Ethereum (smart contract king), and stuff like BNB or Solana.",
            "Bitcoin’s the granddaddy of crypto—digital gold, started by some mystery dude named Satoshi.",
            "Ethereum’s more than money—it runs apps and contracts. Think of it as a blockchain playground!",
            "Hot ones change fast, but BTC and ETH are solid bets. DYOR though—I’m not your financial guru!"
        ]
    },
    {
        "tag": "crypto_trading",
        "patterns": ["How do I trade crypto?", "Any trading tips?", "How do I read crypto charts?"],
        "responses": [
            "Sign up on an exchange like Binance, add some funds, and start small. It’s a wild ride!",
            "Tips? HODL through dips or swing trade if you’re brave. Don’t FOMO into pumps!",
            "Charts show price moves with candlesticks. Green means up, red means ouch—watch the trends!"
        ]
    },
    {
        "tag": "crypto_security",
        "patterns": ["How do I keep my crypto safe?", "What’s a wallet?", "Avoiding crypto hacks"],
        "responses": [
            "Get a hardware wallet, turn on 2FA, and never share your private keys—ever!",
            "A wallet’s where you stash your crypto keys. Hot ones are online; cold ones are offline and safer.",
            "Scammers are everywhere—double-check links, ignore DMs promising riches, and stay sharp."
        ]
    },
    {
        "tag": "crypto_regulations",
        "patterns": ["Is crypto legal?", "What’s the deal with crypto laws?", "Government and crypto?"],
        "responses": [
            "Depends where you are—some places love it, others ban it. Check your local rules!",
            "Laws are a mess right now. Exchanges often need KYC, but it’s still the wild west.",
            "Governments are nervous but curious. More rules are coming, so buckle up!"
        ]
    },
    {
        "tag": "crypto_mining",
        "patterns": ["What’s mining?", "How do I mine crypto?", "Is mining worth it?"],
        "responses": [
            "Mining’s how new coins get made—computers solve puzzles to secure the network.",
            "You need beefy gear like GPUs or ASICs, plus cheap power. It’s a grind!",
            "Worth it? Maybe if your electricity’s free. Otherwise, it’s tough to beat the big miners."
        ]
    },
    {
        "tag": "crypto_wallets",
        "patterns": ["What’s a crypto wallet?", "Best wallet for crypto?", "How do wallets work?"],
        "responses": [
            "It’s where you keep your crypto keys—think of it as your digital piggy bank.",
            "Hardware wallets like Ledger are top-tier for safety. Apps like MetaMask are handy for daily use.",
            "Wallets generate keys: public to receive, private to spend. Lose the private one, and you’re toast!"
        ]
    },
    {
        "tag": "crypto_future",
        "patterns": ["What’s crypto’s future?", "Will crypto take over?", "Where’s crypto headed?"],
        "responses": [
            "Crypto’s going places—more adoption, DeFi, NFTs. It’s just getting started!",
            "Take over? Nah, but it’ll vibe alongside regular money for sure.",
            "Big banks and governments jumping in means it’s here to stay. To the moon, maybe?"
        ]
    },

    # Fun & Casual (Crypto Twist)
    {
        "tag": "jokes",
        "patterns": ["Tell me a joke", "Make me laugh", "Something funny"],
        "responses": [
            "Why’d Bitcoin go to therapy? Too many emotional blocks!",
            "What’s a crypto trader’s favorite dance? The pump and dump!",
            "Why don’t miners date? They’re too busy hashing things out!"
        ]
    },
    {
        "tag": "random_facts",
        "patterns": ["Tell me a crypto fact", "Give me a fun fact", "Surprise me"],
        "responses": [
            "The first Bitcoin buy was two pizzas for 10,000 BTC—worth millions today!",
            "Satoshi Nakamoto’s stash might be over a million BTC, and no one knows who they are!",
            "Ethereum’s name came from a sci-fi book Vitalik Buterin read. Nerdy, huh?"
        ]
    },

    # Miscellaneous Crypto Topics
    {
        "tag": "crypto_news",
        "patterns": ["What’s new in crypto?", "Latest crypto updates?", "Crypto world news?"],
        "responses": [
            "Can’t fetch live news yet, but check CoinDesk or Twitter—crypto moves fast!",
            "Big stuff lately? Think price swings, new coins, or Elon tweeting about Doge again.",
            "Stay tuned to the space—something wild happens every week!"
        ]
    },
    {
        "tag": "crypto_investing",
        "patterns": ["Should I invest in crypto?", "Good time to buy crypto?", "How much to invest?"],
        "responses": [
            "Crypto’s risky but rewarding—just don’t bet the farm, okay?",
            "Timing’s tricky. Dollar-cost averaging beats waiting for the ‘perfect’ dip.",
            "Start small, like 5% of your cash, and learn the ropes. DYOR always!"
        ]
    },
    {
        "tag": "crypto_scams",
        "patterns": ["How do I spot scams?", "Common crypto scams?", "Is this a scam?"],
        "responses": [
            "If it promises ‘guaranteed’ riches, it’s probably a scam. Research everything!",
            "Watch for fake ICOs, phishing sites, and sketchy giveaways. Trust your gut.",
            "Can’t check specifics, but legit projects have real teams and whitepapers—dig in!"
        ]
    },
    {
        "tag": "bitcoin_info",
        "patterns": ["What is Bitcoin?", "Tell me about Bitcoin", "How’s Bitcoin work?"],
        "responses": [
            "Bitcoin’s the first crypto—digital gold since 2009!",
            "It’s decentralized money by some mystery genius.",
            "BTC runs on blockchain—secure and borderless."
        ]
    },
    {
        "tag": "ethereum_info",
        "patterns": ["What is Ethereum?", "Tell me about Ethereum", "Ethereum basics"],
        "responses": [
            "Ethereum’s a platform for apps and smart contracts.",
            "Its coin is Ether—fuel for a blockchain world!",
            "It’s like Bitcoin but with extra superpowers."
        ]
    },
    {
        "tag": "ripple_info",
        "patterns": ["What is Ripple?", "Tell me about Ripple", "How’s Ripple different?"],
        "responses": [
            "Ripple’s fast payment crypto—banks love it!",
            "It’s XRP, built for quick, cheap transfers.",
            "Less decentralized than others, but speedy."
        ]
    },

    # Market and Investment Intents
    {
        "tag": "market_trends",
        "patterns": ["What’s the market like?", "Are prices up?", "How’s crypto doing?"],
        "responses": [
            "Crypto’s wild—check CoinMarketCap for the latest!",
            "Market’s a rollercoaster—up one day, down the next.",
            "Trends? Volatile as always—DYOR!"
        ]
    },
    {
        "tag": "investing",
        "patterns": ["Should I invest?", "Good time to buy?", "How much to invest?"],
        "responses": [
            "Crypto’s risky—only invest what you can lose.",
            "Buy dips if you dare, or average in slowly.",
            "Start small and learn—don’t go all-in!"
        ]
    },
    {
        "tag": "get_price",
        "patterns": ["What’s Bitcoin’s price?", "How much is Ethereum?", "Price of Ripple?"],
        "responses": [
            "No live data yet—check CoinGecko for now!",
            "Prices shift fast—better look online.",
            "Can’t fetch it, but it’s probably pumping or dumping!"
        ]
    },

    # Fun and Casual Intents
    {
        "tag": "fun_facts",
        "patterns": ["Tell me a crypto fact", "Crypto trivia", "Something cool about crypto"],
        "responses": [
            "First Bitcoin buy? Two pizzas for 10,000 BTC!",
            "Satoshi’s identity is still a total mystery.",
            "Dogecoin’s a meme that turned into millions!"
        ]
    },
    {
        "tag": "jokes",
        "patterns": ["Tell me a joke", "Crypto joke", "Make me laugh"],
        "responses": [
            "Why’d Bitcoin cry? Too many dips!",
            "What’s a trader’s pet? A HODL-dog!",
            "Crypto dating tip: Don’t dump on the first pump!"
        ]
    },

    # Fallback Intents
    {
        "tag": "unknown",
        "patterns": ["What?", "Huh?", "I don’t get it"],
        "responses": [
            "Oops, lost me there—try again?",
            "Not sure what you mean—crypto stuff maybe?",
            "Let’s rewind—ask me something else!"
        ]
    },
    {
        "tag": "no_answer",
        "patterns": ["I don’t know", "Not sure", "Can’t help"],
        "responses": [
            "Hmm, stumped me! Let’s talk crypto instead.",
            "No clue there—how about Bitcoin?",
            "I’m out of ideas—try a crypto question!"
        ]
    }
]

In [18]:
# Prepare training data
X = [pattern for intent in intents for pattern in intent["patterns"]]
y = [intent["tag"] for intent in intents for _ in intent["patterns"]]

# Vectorize and train
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)
model = LogisticRegression()
model.fit(X_vec, y)

# Prediction function (example)
def predict_intent(user_input):
    input_vec = vectorizer.transform([user_input])
    tag = model.predict(input_vec)[0]
    for intent in intents:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])
    return "Hmm, not sure about that one. Let’s talk crypto instead!"

# Test it
print(predict_intent("What is cryptocurrency?"))
print(predict_intent("What is bitcoin?"))

It works with a network of computers keeping a public ledger called the blockchain. Pretty slick, right?
Bitcoin’s the first crypto—digital gold since 2009!
